# Type charts

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `repteis` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Compilacao Livros Repteis - 2 a 10 - 2020_04_28.xls'</font>.

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# pacotes para visualização rápida
import seaborn as sns
import matplotlib.pyplot as plt

# pacote para visualização principal
import altair as alt

# habilitando renderizador para notebook
# alt.renderers.enable('notebook')
alt.renderers.enable('default')


# desabilitando limite de linhas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig')

/home/franklin/anaconda3/envs/viz/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<br>

<font size=5>**Paleta de cores por Ordem**</font>

Abaixo está a imagem usada como inspiração (https://color.adobe.com/create/image)

<img src="./src/paleta_cores.jpeg" width='500px'>

Cores: 

- verde_escuro: #284021
- verde_claro: #88BF11
- amarelo: #D9CB0B
- laranja: #D99311
- laranja_escuro: #BF4417
- marrom-_laro: #BF8D7A

In [3]:
# p.s.: caudata is an error and should be removed. 
cores_ordem = {
    'Squamata': '#BF4417',
    'Testudines': '#D9CB0B', 
    'Crocodylia': '#284021'
}

ordens = list(cores_ordem.keys())
cores = list(cores_ordem.values())

<br>


## Graphs

---

### Types (*per year*) per genus

x: Species1, cor: Type Status1, size: counts

In [4]:
# type info is in this column
NewTable['type_status'].value_counts(dropna= False)

NaN          22731
Parátipo       358
Holótipo        32
2005             5
Síntipo          4
Topótipo         1
Lectótipo        1
Name: type_status, dtype: int64

In [5]:
# subsetting
teste = NewTable[['altitude','familia','ordem', 'ano_coleta', 'qualificador_atual', 'numero_catalogo', 
                  'genero_atual', 'especie_atual', 'subespecie_atual', 'type_status']].copy()

# grouping by type, year and order
temp = teste.groupby(['type_status','ano_coleta', 'ordem']).count()['familia'].reset_index().rename(columns={
    'familia':'counts'
})

# p.s.: Cótipo and Topótipo are not types
temp = temp[(temp['type_status'] != 'Cótipo') & (temp['type_status'] != 'Topótipo')]

243 info. de tipos

### Gráf. de Tipos

In [8]:
tipo = alt.Chart(temp, height=150, title='Types per year').mark_circle().encode(
    x = alt.X('ano_coleta:O', title='collected year'),
    y = alt.Y('type_status:N', title= 'type',
              sort=alt.EncodingSortField('tipo', op='count', order='descending')),
    color= alt.Color('ordem', scale=alt.Scale(domain=ordens, range=cores), title='order'), 
    size= alt.Size('counts', scale=alt.Scale()),
    tooltip= [alt.Tooltip('type_status', title='type'),
              alt.Tooltip('ano_coleta', title='collected year'),
              alt.Tooltip('counts', title='counts')]
)

# tipo.save('./graphs/tipo/tipos_por_ano.html')

tipo

alt.Chart(...)

<br>

<font size='6'>Para a base de repteis acaba aqui, por enquanto </font>

<br>

#### defining some parameters

**colors:**
<ul>
    <li style='color:#d62728;'><b> #d62728 </b> - vermelho 1</li>
    <li style='color:#d62729;'><b> #d62729 </b> - vermelho 2</li>
    <li style='color:#f58518;'><b> #f58518 </b> - laranja 1</li>
    <li style='color:#f58519;'><b> #f58519 </b> - laranja 2</li>
    <li style='color:#d95f02;'><b> #d95f02 </b> - laranja escuro</li>
    <li style='color:#4daf4a;'><b> #4daf4a </b> - verde</li>
    <li style='color:#8c6d31;'><b> #8c6d31 </b> - marrom</li>
    <li style='color:#79706e;'><b> #79706e </b> - azul</li>
    <li style='color:#bab0ac;'><b> #bab0ac </b> - azul claro</li>
    <li style='color:#d8b5a5;'><b> #d8b5a5 </b> - bege</li>
    <li style='color:#000000;'><b> #000000 </b> - preto</li>
</ul>

In [66]:
NewTable['type_status'] = NewTable['type_status'].apply(lambda x:str(x))
types = NewTable['type_status'].apply(lambda x:str(x)).unique()

In [67]:
types

array(['nan', 'Holótipo', 'Parátipo', 'Topótipo', 'Lectótipo', 'Síntipo',
       '2005'], dtype=object)

In [40]:
# dictionary with type:color
type_color = {
    'nan':'#000000',      # preto
    'Holótipo':'#d62728',  # vermelho 1
    'Síntipo':'#d62729',   # vermelho 2
    'Parátipo':'#4daf4a',  # verde
    'Topótipo':'#8c6d31',  # marrom
    'Lectótipo':'#f58518', # laranja 1
    '2005':'#79706e',      # azul  OBS: ERRO NA BASE!
}

<br>

### grouping and counting

In [46]:
counts = NewTable.groupby(['determinator_full_name', 'type_status', 'ano_determinacao']).count()['class'].reset_index()
counts.rename(columns={'class':'counts'}, inplace=True)

## types and non-types per determiner

In [59]:
g1 = alt.Chart(counts[counts['type_status'] != 'nan'], title='Tipos por Determinador', 
               height=300, width=400).mark_circle().encode(
    x = alt.X('ano_determinacao:O', title='Ano de Determinação'),
    y = alt.Y('determinator_full_name:N', title='', 
             sort= alt.EncodingSortField(field= 'counts', op='sum', order='descending')),
    color = alt.Color('type_status:N', title='Tipo', 
                     scale = alt.Scale(domain=list(type_color.keys()), range= list(type_color.values()))), 
    size = alt.Size('counts:Q'), 
    tooltip = alt.Tooltip(['determinator_full_name', 'type_status', 'ano_determinacao', 'counts'])
)

# saving graph
g1.save('./graphs/tipo/tipos_por_determinador.html')


g1

alt.Chart(...)

In [65]:
g1 = alt.Chart(counts[counts['type_status'] == 'nan'], title='Não-tipos por Determinador', 
               height=1000, width=400).mark_circle(color='#000000').encode(
    x = alt.X('ano_determinacao:O', title='Ano de Determinação'),
    y = alt.Y('determinator_full_name:N', title='', 
             sort= alt.EncodingSortField(field= 'counts', op='sum', order='descending')),
    size = alt.Size('counts:Q'), 
    tooltip = alt.Tooltip(['determinator_full_name', 'ano_determinacao', 'counts'])
)

# saving graph
# g1.save('./graphs/tipo/nao-tipos_por_determinador.html')

# g1

<br>

## types per collector

-----

### grouping and counting

In [89]:
counts = NewTable.groupby(['collector_full_name', 'type_status', 'ano_coleta']).count()['class'].reset_index()
counts.rename(columns={'class':'counts'}, inplace=True)
counts.sort_values('counts', inplace=True, ascending=False)

In [91]:
g1 = alt.Chart(counts[counts['type_status'] != 'nan'], title='Tipos por Coletor', 
               height=700, width=400).mark_circle().encode(
    x = alt.X('ano_coleta:O', title='Ano de Determinação'),
    y = alt.Y('collector_full_name:N', title='', 
             sort= alt.EncodingSortField(field= 'counts', op='sum', order='descending')),
    color = alt.Color('type_status:N', title='Tipo', 
                     scale = alt.Scale(domain=list(type_color.keys()), range= list(type_color.values()))), 
    size = alt.Size('counts:Q'), 
    tooltip = alt.Tooltip(['collector_full_name', 'type_status', 'ano_coleta', 'counts'])
)

# saving graph
# g1.save('./graphs/tipo/tipos_por_coletor.html')

g1

alt.Chart(...)

In [95]:
g1 = alt.Chart(counts[counts['type_status'] == 'nan'].iloc[0:200], title='Não-tipos por Coletor', 
               height=700, width=400).mark_circle(color= '#000000').encode(
    x = alt.X('ano_coleta:O', title='Ano de Determinação'),
    y = alt.Y('collector_full_name:N', title='', 
             sort= alt.EncodingSortField(field= 'counts', op='sum', order='descending')),
    size = alt.Size('counts:Q'), 
    tooltip = alt.Tooltip(['collector_full_name', 'type_status', 'ano_coleta', 'counts'])
)

# saving graph
# g1.save('./graphs/tipo/nao-tipos_por_coletor.html')

g1

alt.Chart(...)

<br>

**The end!**

-----